In [ ]:
# https://github.com/alexnguyen9/recipe-matcher

In [110]:
import pandas as pd
import pickle
from flask import Flask, request, render_template, url_for, session
import numpy as np
from scipy.spatial.distance import cdist
import re
import boto3

In [111]:
food_matrix = pickle.load(open('food_matrix.pkl','rb'))
transformer = pickle.load(open('transformer.pkl','rb'))
food_data = pickle.load(open('main_data.pkl','rb'))

In [ ]:
def predict(ingredients):

    
    
    output = str(ingredients)
    
    y = transformer.transform([output])

    m = cdist(y.toarray()[0].reshape(1,-1), food_matrix, metric='cosine')
    global index 
    index = np.argsort(m[0]).tolist()
    counter = 0
    
    topNReciepes = 3
    for i in range(3):
        instructions = food_data.instructions[index[i]]
        title = food_data.title[index[i]]
        ingredients = food_data.ingredients[index[i]]
        source = food_data.url[index[i]]
    
        print ("title: ", title)
        print ("ingredients: ", ingredients)
        print ("--------------------")


In [123]:
ingredients = []
current = ""
while (current != 'end'):
    current = input("Enter a ingredient: ")
    ingredients.append(current)

print("Fetching prediction")
print (predict(ingredients))

Enter a ingredient: jalapeno
Enter a ingredient: tomato
Enter a ingredient: cheese
Enter a ingredient: yeast
Enter a ingredient: end
Fetching prediction
title:  Taco Rice Bowl
ingredients:  ['1 pound ground beef', '1 cup shredded Cheddar cheese', '1 cup chopped tomato']
--------------------
title:  Cheesy Pepperoni Parmesan Pizza
ingredients:  ['5 cups all-purpose flour, divided', '1 cup Kraft® Grated Parmesan Cheese', '1 teaspoon fast-rising dry yeast', '1/2 teaspoon salt', '2 cups cold water', '1/4 cup olive oil', "1/2 cup Hunt's® Diced Tomatoes with Basil, Garlic, and Oregano", "1 (14.5 ounce) can Hunt's® Diced Tomatoes, drained", '2 cups Kraft® Shredded Mozzarella Cheese', '10 slices pepperoni', '1 tablespoon Kraft® Grated Parmesan Cheese']
--------------------
title:  Taco Waffles from Eggland's Best
ingredients:  ["2 Eggland's Best Eggs, large", '1/4 cup bacon bits', '2 jalapeno chile peppers, seeded and diced', '1/4 cup shredded Mexican cheese blend', '2 tablespoons chopped fres

# Shipment

<!--youtube  -->
https://www.youtube.com/watch?v=C_AtgCm43Nk <br>
https://github.com/learn-mikegchambers-com/aws-mls-c01/tree/master/8-SageMaker/SageMaker-Script-Mode

In [76]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
import boto3
import os

In [101]:
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()

s3_prefix = "halifaxfoodie-ml-data"
pickle_s3_prefix = f"{s3_prefix}/pkl"
pickle_s3_uri = f"s3://{bucket}/{s3_prefix}/pkl"
pickle_train_s3_uri = f"{pickle_s3_uri}/food_matrix.pkl"

train_dir = os.path.join(os.getcwd(), "")
print (train_dir)
print (pickle_train_s3_uri)

/home/ec2-user/SageMaker/
s3://sagemaker-us-east-1-585512233622/halifaxfoodie-ml-data/pkl/food_matrix.pkl


In [100]:
s3_resource_bucket = boto3.Session().resource("s3").Bucket(bucket)
s3_resource_bucket.Object(os.path.join(pickle_s3_prefix, "food_matrix.pkl")).upload_file(
    train_dir + "/food_matrix.pkl"
)
print (s3_resource_bucket)

s3.Bucket(name='sagemaker-us-east-1-585512233622')


In [79]:
hyperparameters = {
    "copy_X": True,
    "fit_intercept": True,
    "normalize": False,
}

In [102]:
train_instance_type = "ml.m5.large"

inputs = {
    "train": pickle_train_s3_uri
}

In [105]:
estimator_parameters = {
    "entry_point": "script.py",
    "source_dir": "scripts",
    "framework_version": "0.23-1",
    "py_version": "py3",
    "instance_type": train_instance_type,
    "instance_count": 1,
    "hyperparameters": hyperparameters,
    "role": role,
    "base_job_name": "similarity-model",
}

estimator = SKLearn(**estimator_parameters)


In [ ]:
# estimator.fit(inputs)
estimator.fit(inputs)